# Intro
123

### TOC
[123](123)

In [ ]:
%load_ext autoreload
%autoreload 2
import os
os.chdir("/Users/markusdollmann/Desktop/eliiza/cv_cookie_cutter/")

In [ ]:
import tensorflow as tf

tf.random.stateless_uniform(
    [5],
    (42,42),
    minval=0,
    maxval=10,
)


In [ ]:
for _ in range(5):
  print(tf.random.stateless_uniform(
    [10],
    (42,42),
    minval=0,
    maxval=10,
)
)

# ALL TO IMPORT FROM import tensorflow_models.vision as tmv.augment !!!

In [ ]:
def _convert_translation_to_transform(translations: tf.Tensor) -> tf.Tensor:
  """Converts translations to a projective transform.
  The translation matrix looks like this:
    [[1 0 -dx]
     [0 1 -dy]
     [0 0 1]]
  Args:
    translations: The 2-element list representing [dx, dy], or a matrix of
      2-element lists representing [dx dy] to translate for each image. The
      shape must be static.
  Returns:
    The transformation matrix of shape (num_images, 8).
  Raises:
    `TypeError` if
      - the shape of `translations` is not known or
      - the shape of `translations` is not rank 1 or 2.
  """
  translations = tf.convert_to_tensor(translations, dtype=tf.float32)
  if translations.get_shape().ndims is None:
    raise TypeError('translations rank must be statically known')
  elif len(translations.get_shape()) == 1:
    translations = translations[None]
  elif len(translations.get_shape()) != 2:
    raise TypeError('translations should have rank 1 or 2.')
  num_translations = tf.shape(translations)[0]

  return tf.concat(
      values=[
          tf.ones((num_translations, 1), tf.dtypes.float32),
          tf.zeros((num_translations, 1), tf.dtypes.float32),
          -translations[:, 0, None],
          tf.zeros((num_translations, 1), tf.dtypes.float32),
          tf.ones((num_translations, 1), tf.dtypes.float32),
          -translations[:, 1, None],
          tf.zeros((num_translations, 2), tf.dtypes.float32),
      ],
      axis=1,
    )

def translate(image: tf.Tensor, translations) -> tf.Tensor:
  """Translates image(s) by provided vectors.
  Args:
    image: An image Tensor of type uint8.
    translations: A vector or matrix representing [dx dy].
  Returns:
    The translated version of the image.
  """
  transforms = _convert_translation_to_transform(translations)
  return transform(image, transforms=transforms)

In [ ]:
def _convert_angles_to_transform(angles: tf.Tensor, image_width: tf.Tensor,
                                 image_height: tf.Tensor) -> tf.Tensor:
  """Converts an angle or angles to a projective transform.
  Args:
    angles: A scalar to rotate all images, or a vector to rotate a batch of
      images. This must be a scalar.
    image_width: The width of the image(s) to be transformed.
    image_height: The height of the image(s) to be transformed.
  Returns:
    A tensor of shape (num_images, 8).
  Raises:
    `TypeError` if `angles` is not rank 0 or 1.
  """
  angles = tf.convert_to_tensor(angles, dtype=tf.float32)
  if len(angles.get_shape()) == 0:  # pylint:disable=g-explicit-length-test
    angles = angles[None]
  elif len(angles.get_shape()) != 1:
    raise TypeError('Angles should have a rank 0 or 1.')
  x_offset = ((image_width - 1) -
              (tf.math.cos(angles) * (image_width - 1) - tf.math.sin(angles) *
               (image_height - 1))) / 2.0
  y_offset = ((image_height - 1) -
              (tf.math.sin(angles) * (image_width - 1) + tf.math.cos(angles) *
               (image_height - 1))) / 2.0
  num_angles = tf.shape(angles)[0]
  return tf.concat(
      values=[
          tf.math.cos(angles)[:, None],
          -tf.math.sin(angles)[:, None],
          x_offset[:, None],
          tf.math.sin(angles)[:, None],
          tf.math.cos(angles)[:, None],
          y_offset[:, None],
          tf.zeros((num_angles, 2), tf.dtypes.float32),
      ],
      axis=1,
  )

def rotate(image: tf.Tensor, degrees: float) -> tf.Tensor:
  """Rotates the image by degrees either clockwise or counterclockwise.
  Args:
    image: An image Tensor of type uint8.
    degrees: Float, a scalar angle in degrees to rotate all images by. If
      degrees is positive the image will be rotated clockwise otherwise it will
      be rotated counterclockwise.
  Returns:
    The rotated version of image.
  """
  # Convert from degrees to radians.
  degrees_to_radians = math.pi / 180.0
  radians = tf.cast(degrees * degrees_to_radians, tf.float32)

  original_ndims = tf.rank(image)
  image = to_4d(image)

  image_height = tf.cast(tf.shape(image)[1], tf.float32)
  image_width = tf.cast(tf.shape(image)[2], tf.float32)
  transforms = _convert_angles_to_transform(
      angles=radians, image_width=image_width, image_height=image_height)
  # In practice, we should randomize the rotation degrees by flipping
  # it negatively half the time, but that's done on 'degrees' outside
  # of the function.
  image = transform(image, transforms=transforms)
  return from_4d(image, original_ndims)

In [ ]:
def cutout(image: tf.Tensor, pad_size: int, replace: int = 0) -> tf.Tensor:
  """Apply cutout (https://arxiv.org/abs/1708.04552) to image.
  This operation applies a (2*pad_size x 2*pad_size) mask of zeros to
  a random location within `image`. The pixel values filled in will be of the
  value `replace`. The location where the mask will be applied is randomly
  chosen uniformly over the whole image.
  Args:
    image: An image Tensor of type uint8.
    pad_size: Specifies how big the zero mask that will be generated is that is
      applied to the image. The mask will be of size (2*pad_size x 2*pad_size).
    replace: What pixel value to fill in the image in the area that has the
      cutout mask applied to it.
  Returns:
    An image Tensor that is of type uint8.
  """
  if image.shape.rank not in [3, 4]:
    raise ValueError('Bad image rank: {}'.format(image.shape.rank))

  if image.shape.rank == 4:
    return cutout_video(image, replace=replace)

  image_height = tf.shape(image)[0]
  image_width = tf.shape(image)[1]

  # Sample the center location in the image where the zero mask will be applied.
  cutout_center_height = tf.random.uniform(
      shape=[], minval=0, maxval=image_height, dtype=tf.int32)

  cutout_center_width = tf.random.uniform(
      shape=[], minval=0, maxval=image_width, dtype=tf.int32)

  image = _fill_rectangle(image, cutout_center_width, cutout_center_height,
                          pad_size, pad_size, replace)

  return image

In [ ]:
import tensorflow_models.vision as tmv

banana = [
  tmv.augment.solarize, 
  tmv.augment.solarize_add,
  tmv.augment.color,
  tmv.augment.contrast,
  tmv.augment.brightness,
  tmv.augment.posterize,
  tmv.augment.wrapped_rotate,
  tmv.augment.shear_x,
  tmv.augment.shear_y,
  tmv.augment.autocontrast,
  tmv.augment.sharpness,
  tmv.augment.equalize,
  tmv.augment.invert,
]

In [ ]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
np.random.seed(42)
xs = list(range(50))
np.random.seed(42)
ys = np.random.normal(loc=1.5,scale=0.5,size=len(xs))
# ys[12]+= 4
# ys[27]+= 4
fig = px.scatter(title="Low Signal to Noise")
# fig.update_traces(marker=dict(size=12,))
fig.add_trace(go.Scatter(x=xs, y=ys,
                    mode='lines',
                    name='fit'))

# fig.add_trace(go.Scatter(x=[1,3], y=[1.5,3.5],
#                     mode='lines',
#                     line={'dash': 'dash', 'color': 'green'},
#                     name='upper bound'))

# fig.add_trace(go.Scatter(x=[1.5,3], y=[0.9,2.7],
#                     mode='lines',
#                     line={'dash': 'dash', 'color': 'green'},
#                     name='lower bound'))

# fig.add_trace(go.Scatter(x=[1.5],y=[1.5,2.5],name="data", mode="markers",marker=dict(size=8,)))
# fig.update_layout(yaxis_range=[0.9,3.5])

In [ ]:
# import numpy as np
import plotly.express as px
import plotly.graph_objects as go

fig = px.scatter(title="Not Enough Data")
fig.update_traces(marker=dict(size=12,))
fig.add_trace(go.Scatter(x=[1,3], y=[1,3],
                    mode='lines',
                    name='fit 1'))

fig.add_trace(go.Scatter(x=[1,3], y=[3.1,0.9],
                    mode='lines',
                    #line={'dash': 'dash', 'color': 'green'},
                    name='fit 2'))

fig.add_trace(go.Scatter(x=[1.625,2.5], y=[0.9,3.5],
                    mode='lines',
                    #line={'dash': 'dash', 'color': 'green'},
                    name='fit 3'))

fig.add_trace(go.Scatter(x=[2],y=[2],name="data", mode="markers",marker=dict(size=8,)))
fig.update_layout(yaxis_range=[0.9,3.5],xaxis_range=[1,3])

In [ ]:
import pandas as pd
pd.set_option("display.max_row",104)
a = pd.read_excel("~/Desktop/labels_PRS_pressure.xlsx")

In [ ]:
a["PRS#"].value_counts()

In [ ]:
set(a["PRS#"].tolist()).symmetric_difference(list(range(0,150)))

In [ ]:
import tensorflow as tf
alpha = 1
width = 3
pd.Series((tf.random.gamma(shape=(width*500000,),alpha=1,beta=25)).numpy()).hist(bins=50,log=True)

In [ ]:
import numpy as np
pd.Series(np.random.dirichlet([alpha] * width*500).astype(np.float32)).hist(bins=50)